In [24]:
import json
import spacy
import yaml
import os
import operator
import copy
from spacy.vocab import Vocab
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def import_bios(path):
    with open(path) as fp:
        temp = json.load(fp)
        ids = [i['_id'] for i in temp['results'] if i['bio']]
        bio = [i['bio'] for i in temp['results'] if i['bio']]
        return ids,bio

In [ ]:
def export_scores(score,path,bios,id):
    result = {}
    

In [42]:
def lemma_sw_dict(bios):
    nlp = spacy.load('en')
    words = set(nltk.corpus.words.words())
    stop_words = set(stopwords.words('english'))
    clean = bios
    for i, doc in enumerate(clean):
        doc = nlp(doc)
        clean[i] = " ".join([token.lemma_ for token in doc if (str(token.lemma_) in words and str(token.lemma_) not in stop_words)])
    return clean

In [4]:
def tfidf(bios):
    vectorizer = TfidfVectorizer(smooth_idf= True)
    X = vectorizer.fit_transform(bios)
    feature_names = vectorizer.get_feature_names()
    vec = X.toarray()
    return vec,feature_names

In [47]:
def tf_idf_score(ids,vec):
    score = []
    for i,j in zip(ids,vec):
        score.append((i, sum(j)/len(j)))
    score.sort(key=operator.itemgetter(1), reverse=True)
    return score

In [194]:
a = []
for i,j in zip(ids,sc):
    a.append((i,j))
a.sort(key=operator.itemgetter(1), reverse=True)
a

[('5cf71b9b8b4a5715002aca18', 0.022600073645121878),
 ('5cf72619a3082b1f0009c394', 0.021114224016306495),
 ('5c8e94e89f217f1100d783c1', 0.02089446130235234),
 ('594c6927d108c9e056aeec5a', 0.018601455158944052),
 ('5c0041ff5fbacf94336293b6', 0.017967486705502158),
 ('5cf724ea1e6ced1600ff37ad', 0.017833084889826652),
 ('5c591ccc36c02411007fc7b9', 0.017786904366475606),
 ('5cce07bb6c90731500da001c', 0.016780585503191014),
 ('5cf7116ed2e6c41500ba5d28', 0.015916070624205133),
 ('55bfe2c7fb5fe72d158aa394', 0.013917555356722604),
 ('5bd630f84fa083ae3c79ab52', 0.013909672619269087),
 ('5bc96fa75dbb0d79393c3269', 0.013870047366333791),
 ('5bc96fa75dbb0d79393c3269', 0.013870047366333791),
 ('5bc96fa75dbb0d79393c3269', 0.013870047366333791),
 ('5bc96fa75dbb0d79393c3269', 0.013870047366333791),
 ('5cf6f7d32e637d16001cf214', 0.013840486733806329),
 ('5408bc17d402808f7215670c', 0.013776287072129528),
 ('54853dcb4c9ad4ec3859c8f5', 0.013134329788292405),
 ('5cf706d86968511700f18db2', 0.012660522890534

In [48]:
if __name__ == '__main__':
    with open('../../../config/tinder.yaml', 'r') as ymlfile:
        cfg = yaml.load(ymlfile)
    path = cfg['output_folder']+cfg['city']+'/'
    json_files = [pos_json for pos_json in os.listdir(path) if pos_json.endswith('.json')]
    data_path = ('../../../output/data')
    if not os.path.exists(data_path):
            os.mkdir(data_path)

    for file in json_files:
        ids,bios = import_bios(path+json_files[0])
#         old_bios = bios
        cleaned = lemma_sw_dict(bios)
#         print(cleaned)
        vector,terms =tfidf(cleaned)
#         print(bios)
#         print(vector)
        scores = tf_idf_score(ids,vector)
        print(scores)
#         export(scores,op_path)
    

/home/pallavi/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


[('5cf71b9b8b4a5715002aca18', 0.020477482243935658), ('5c8e94e89f217f1100d783c1', 0.01823897706962276), ('5c591ccc36c02411007fc7b9', 0.015647513948878512), ('5cf72619a3082b1f0009c394', 0.014895997114799423), ('594c6927d108c9e056aeec5a', 0.014785615143428339), ('5cf724ea1e6ced1600ff37ad', 0.01333380328458161), ('5cf7116ed2e6c41500ba5d28', 0.01312404386629623), ('5c0041ff5fbacf94336293b6', 0.012855134846338762), ('5bd630f84fa083ae3c79ab52', 0.012252443985405953), ('5c05ff239496ea0733dd24bb', 0.012228736038164288), ('5bc96fa75dbb0d79393c3269', 0.012208998903367206), ('5bc96fa75dbb0d79393c3269', 0.012208998903367206), ('5bc96fa75dbb0d79393c3269', 0.012208998903367206), ('5bc96fa75dbb0d79393c3269', 0.012208998903367206), ('5cce07bb6c90731500da001c', 0.01197379370871508), ('55bfe2c7fb5fe72d158aa394', 0.011619630087961763), ('5cf740166968511700f1988a', 0.011585533369724513), ('5408bc17d402808f7215670c', 0.011539575083392489), ('5cf6f7d32e637d16001cf214', 0.011096872973826557), ('5cf724d510d71

In [ ]:
_id,b = 

In [231]:
 l = []
for bio,ids in zip(b,i):
    l.append((ids,bio))
d = {}
d.update(l)

In [230]:
path = '/home/pallavi/Project/SmartDate/output/Boston/2019-06-05.json'
i,b = import_bios(path)

In [232]:
d['']

{'511eb37131d9a93d19000306': 'Only here for your dogs  \nMA-->ME\nIG: hannahendricks_',
 '5408bc17d402808f7215670c': 'I like people who message first. Tell me about your day or a funny story or something.',
 '54853dcb4c9ad4ec3859c8f5': "“Don't make smoothies and expect the boys to come to the yard”- my mom wouldn’t let me make this my senior quote \n\n@savannaVjohnson",
 '55bfe2c7fb5fe72d158aa394': 'Figure skater. Rugby player. Pet enthusiast.\nLow maintenance. High quality.\n🇷🇺🇮🇪',
 '55eb7a350029196a74aee612': 'BSN\nlife is lit',
 '564b50dfc57146f613d51f55': '#callherdaddy',
 '594c6927d108c9e056aeec5a': 'Hi! Looking for some cool people to spend time with and smoke. I love to do anything outside but obviously love watching Netflix on rainy summer nights. HMU and lets smoke',
 '59f4b91bd6967c4b6ba5f3aa': 'Here for whatever my dude I just wanna have fun this summer\npsych major 💕',
 '5adfb8a0d93e256a0415082c': '♏️\nBSU ‘19\n🇮🇹🇮🇪\nig: morganviall',
 '5b0c42e9288cb58f377120a3': 'Kimmauric

In [208]:
bios

['fitness',
 'like big lie',
 '',
 'hockey',
 'cool could spell give five',
 'plus deal breaker respect',
 'ah go life short prefer fun time long time snap lucky',
 'like people message first day funny story something',
 'probably show u usually never text first snap',
 'dumb another country June',
 'Boston graduate dentist like go twin sister',
 '',
 'reason',
 '',
 'something serious know art car accident',
 'r gluten free',
 '',
 'go outside good time',
 'go date',
 'dog',
 'favorite type girl like romantic low ask furry morbidly obese',
 '',
 'warm heart dog name Winston ya',
 'Boston graduate dentist like go twin sister',
 'someone fun',
 'dog plus',
 'reason',
 'least double use work gym raise Vermont work right cartoon dinosaur tinder cat need name Wednesday like Wednesday sleep',
 'September',
 'whatever dude fun summer psych major',
 'cool cool doubt doubt',
 'good time long time',
 'life light',
 'whiskey ice bottom shelf cook capable unwilling clean foot absolute unit',
 'du

In [225]:
s = 'sums'
s = nlp(s)
for i in s:
    print(i.lemma_)

sum


In [245]:

d[a[20][0]]


'Here for whatever my dude I just wanna have fun this summer\npsych major 💕'

In [246]:
a[20][1]

0.012655668061579724